# Was sagt ein positver Corona-Test aus?



In [ ]:
!pip install altair 

In [360]:
import numpy as np
from ipywidgets import interact
import altair as alt
import pandas as pd
import scipy.stats
import logging

logger = logging


def normal_dens(mu, sigma, x):
    return (
        1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
    )


def epan_kernel(u, height):
    return height * max(0, 3.0 / 4 * (1 - (u) ** 2))


@interact(
    inzidenz=(1, 100_000),
    sensitivity=(0.0, 0.99, 0.01),
    specificity=(0.0, 0.99, 0.01),
    group_size=(1, 100_000),
)
def my_function(inzidenz=15000, sensitivity=0.97, specificity=0.99, group_size=5_000):

    # inzident bedeutet infizierte je 100.000 Einwohner. 
    # Richtigerweise müssten wir die tatsächliche Anzahl infizierter wissen, 
    # da dies jedoch unbekannt ist nehmen wir die Inzidenz als Schätzer.
    
    anteil_positiv = inzidenz / 100_000

    # => tp je 100_000 Einwohner
    p = anteil_positiv * group_size
    n = group_size - p

    tp = sensitivity * p
    tn = specificity * n

    fn = p - tp
    fp = n - tn

    def f_sensitivity(tp, fn):
        return tp / (tp + fn)

    def f_specificity(tn, fp):
        return tn / (tn + fp)

    axis2 = np.linspace(-6, 6, num=128)

    shift_fn = -scipy.stats.norm.ppf(sensitivity)
    shift_fp = -scipy.stats.norm.ppf(1 - specificity)

    p_chart = (
        alt.Chart(
            pd.DataFrame(
                {"x": axis2, "y": anteil_positiv * normal_dens(0, 1, axis2 - shift_fn)}
            )
        )
        .mark_area(opacity=0.5, color="red")
        .encode(x="x", y="y")
    )

    fn_chart = (
        alt.Chart(
            pd.DataFrame(
                {
                    "x": axis2,
                    "y": anteil_positiv
                    * normal_dens(0, 1, axis2 - shift_fn)
                    * (axis2 > 0),
                }
            )
        )
        .mark_area(opacity=0.5, color="darkred")
        .encode(x="x", y="y")
    )

    n_chart = (
        alt.Chart(
            pd.DataFrame(
                {
                    "x": axis2,
                    "y": (1 - anteil_positiv) * normal_dens(0, 1, axis2 - shift_fp),
                }
            )
        )
        .mark_area(opacity=0.5, color="blue")
        .encode(x="x", y="y")
    )

    fp_chart = (
        alt.Chart(
            pd.DataFrame(
                {
                    "x": axis2,
                    "y": (1 - anteil_positiv)
                    * normal_dens(0, 1, axis2 - shift_fp)
                    * (axis2 < 0),
                }
            )
        )
        .mark_area(opacity=0.5, color="darkblue")
        .encode(
            alt.X("x", axis=alt.Axis(title="Maßeinheit des Tests")),
            alt.Y("y", axis=alt.Axis(title="Anteil")),
        )
    )

    cutoff = alt.Chart(pd.DataFrame({"x": [0]})).mark_rule().encode(x="x")

    (p_chart + fn_chart + n_chart + fp_chart + cutoff).display()

    logger.debug("p " + str(p))
    logger.debug("n " + str(n))
    logger.debug("anteil " + str(anteil_positiv))
    logger.debug("FP: " + str(fp))
    logger.debug("FN: " + str(fn))
    logger.debug("TP: " + str(tp))
    logger.debug("TN: " + str(tn))
    logger.debug("TP: " + str(tp))
    logger.debug("TN: " + str(tn))
    logger.debug(12 * p * np.mean(normal_dens(0, 1, axis2 - shift_fn) * (axis2 > 0)))
    logger.debug(12 * n * np.mean(normal_dens(0, 1, axis2 - shift_fp) * (axis2 < 0)))

    print(
        "Von "
        + str(group_size)
        + " getestet Personen bei einer Inzidenz von "
        + str(int(inzidenz))
        + " erwarten wir dass mindestens "
        + str(int(np.floor(fn)))
        + " Tests negativ sind obwohl die getesteten Personen positv sind."
    )
    print(
        "Von "
        + str(group_size)
        + " getestet Personen bei einer Inzidenz von "
        + str(int(inzidenz))
        + " erwarten wir dass mindestens "
        + str(int(np.floor(fp)))
        + " Tests positiv sind obwohl die getesteten Personen negativ sind."
    )

    praevalenz = anteil_positiv
    ppv = (senistivity * praevalenz) / (
        senistivity * praevalenz + (1 - specificity) * (1 - praevalenz)
    )
    npv = (specificity * (1 - praevalenz)) / (
        specificity * (1 - praevalenz) + (1 - senistivity) * praevalenz
    )

    print(
        "Ein positiv getesterer ist mit "
        + str(np.round(ppv * 100, 3))
        + " % wirklich positiv."
    )
    print(
        "Ein negativ getesterer ist mit "
        + str(np.round(npv * 100, 3))
        + " % wirklich negativ."
    )


interactive(children=(IntSlider(value=15000, description='inzidenz', max=100000, min=1), FloatSlider(value=0.9…